In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')

In [2]:
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
import torch


processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
  "Qwen/Qwen2.5-VL-3B-Instruct",
  torch_dtype=torch.bfloat16,
  device_map='auto',
  attn_implementation='flash_attention_2'
)

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from qwenvl.data.utils import make_model_input

conversation = [
  {"role": "user",
   "content": [
    {
      'type': 'text',
      'text': "What is this logo for?"
    },
    {
      'type': 'image',
      'image': 'python_logo.png'
    }
    ]}
]
processor.image_processor.max_pixels = 20000
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
print(text)
inputs = make_model_input(conversation, processor, None, False)
inputs['input_ids'].shape

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is this logo for?<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant



torch.Size([1, 47])

In [23]:
# ...existing code...

import torch

from transformers.modeling_outputs import BaseModelOutputWithPast

def decode_tokens(tokenizer, tensor):
  if tensor is None:
    return None
  if tensor.ndim > 1:
    tensor = tensor[0]
  return tokenizer.decode(tensor.tolist(), skip_special_tokens=True)


# Prepare storage for hooks
record = {}

def get_output_hook(name):
  def hook(module, input, output):
    if isinstance(output, BaseModelOutputWithPast):
      output = output['last_hidden_state']
      
    record[f"{name}_output"] = output.detach().cpu()
  return hook


# Attach hooks
lang_hook = model.language_model.register_forward_hook(
    get_output_hook("language_model")
)
vis_hook = model.visual.register_forward_hook(
    get_output_hook("visual")
)

# Run forward pass
try:
  with torch.no_grad():
    outputs = model(**inputs)
except Exception as e:
  raise e
finally:
  # Remove hooks
  lang_hook.remove()
  vis_hook.remove()

# Get logits and decode output tokens
logits = outputs.logits
output_ids = logits.argmax(-1)
decoded_output = decode_tokens(processor.tokenizer, output_ids)

In [11]:
outputs['logits'].shape

torch.Size([1, 47, 151936])

In [33]:
image_pad_token = processor.image_token_id
img_token_locs = torch.where(inputs['input_ids'] == image_pad_token)
img_token_locs = (img_token_locs[0][:-1], img_token_locs[1][:-1])

In [37]:
# Get the image tokens from the inputs
img_predict = record['language_model_output'][img_token_locs]
img_targets = record['visual_output'][1:]

In [39]:
torch.nn.functional.cosine_similarity(
    img_predict, img_targets, dim=-1
).mean().item()

0.06591796875

In [ ]:
token_predict = record['language_model_output']